In [21]:
import pandas as pd

import gensim

import string
nltk.download('punkt')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_json("/content/Cell_Phones_and_Accessories_5.json", lines = True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [7]:
df.shape

(194439, 9)

In [35]:
# Apply inbuilt preprocess model. # gensim is DL library
# follows the basic approach using nltk for pre-processing RE, split, lammetization.
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)

In [36]:
review_text

,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."
...,...
194434,"[works, great, just, like, my, original, one, ..."
194435,"[great, product, great, packaging, high, quali..."
194436,"[this, is, great, cable, just, as, good, as, t..."
194437,"[really, like, it, becasue, it, works, well, w..."


In [34]:
# Pre-processing using manual approach
# def preprocess(text):
#   text = text.lower()
#   text = ''.join([i for i in text if i not in string.punctuation])
#   text = ' '.join([word for word in nltk.word_tokenize(text) if word not in set(stopwords.words('english'))])
#   text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])
#   text = nltk.word_tokenize(text)
#   return text

# review_text = df['reviewText'].apply(lambda x: preprocess(x))
# review_text

In [40]:
df["reviewText"][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [38]:
print(review_text[0])

['they', 'look', 'good', 'and', 'stick', 'good', 'just', 'don', 'like', 'the', 'rounded', 'shape', 'because', 'was', 'always', 'bumping', 'it', 'and', 'siri', 'kept', 'popping', 'up', 'and', 'it', 'was', 'irritating', 'just', 'won', 'buy', 'product', 'like', 'this', 'again']


In [41]:
# We build our own model (Self-Trained Model)
model = gensim.models.Word2Vec(window=10, min_count=2, sg=0) # sg=0 means CBOW

In [42]:
# Model building
model.build_vocab(review_text, progress_per=1000)

In [43]:
model.train(review_text, total_examples = model.corpus_count, epochs = model.epochs)

(61508463, 83868975)

In [72]:
model.wv['smartphone']

array([-0.4973157 ,  1.0760666 , -0.9720773 , -2.1608918 , -2.9978824 ,
       -0.09983324,  1.1617098 ,  1.3207566 ,  0.09066898, -0.08511674,
        2.400277  ,  0.04521139,  1.5050489 ,  3.2684848 ,  2.2322354 ,
       -3.3943605 ,  1.0248263 , -0.10700987,  3.057251  , -0.14298947,
        0.44220996, -2.4612186 ,  1.0100083 , -0.9345213 , -1.2648103 ,
       -1.5309843 ,  1.2090155 ,  0.61000127,  0.24319258,  2.4077382 ,
       -2.875179  , -1.7750609 ,  0.29704314,  2.4248822 ,  2.6053793 ,
        0.47081465, -0.51942456,  0.6478804 , -3.4285781 ,  1.0737709 ,
        1.1186848 , -1.3889842 ,  0.19099829,  2.7071283 ,  1.0463866 ,
        0.09068798, -1.7620851 ,  1.6608441 , -0.9636474 ,  0.10032482,
        1.2840432 , -0.08142482,  0.9432732 , -4.8597946 ,  2.8935525 ,
        1.2894092 , -1.9321474 ,  2.729742  , -0.1523499 ,  2.870723  ,
        0.07041281,  0.56460255,  1.3070611 ,  0.64782035,  0.11136958,
       -0.6510255 ,  0.46282968,  0.85908526, -0.87381834,  2.16

In [73]:
len(model.wv['phone'])

100

In [59]:
model.wv.most_similar('device')

[('unit', 0.6765721440315247),
 ('devices', 0.6223770380020142),
 ('smartphone', 0.6126648783683777),
 ('source', 0.6037387251853943),
 ('hub', 0.5663594007492065),
 ('headset', 0.5661579370498657),
 ('computer', 0.563044548034668),
 ('zomm', 0.5414212346076965),
 ('pc', 0.5396284461021423),
 ('system', 0.533367395401001)]

In [58]:
model.wv.most_similar('phone')

[('iphone', 0.5690363645553589),
 ('cellphone', 0.5337002277374268),
 ('lap', 0.5097962021827698),
 ('it', 0.4986473321914673),
 ('case', 0.46878165006637573),
 ('device', 0.45130762457847595),
 ('shoulder', 0.4302210211753845),
 ('gn', 0.4232967495918274),
 ('hand', 0.41643112897872925),
 ('clumsiness', 0.41168129444122314)]

In [46]:
model.wv.similarity('cheap', 'expensive')

0.45244032

In [61]:
model.wv.similarity('smartphone', 'iphone')

0.40552858

In [60]:
model.wv.doesnt_match(['smartphone', 'phone', 'car'])

'car'